In [3]:
import torch
import torch.nn.functional as F

# Import data
names = open('names.txt', 'r').read().splitlines()
chars = sorted(list(set(''.join(names))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i+1:s for i,s in enumerate(chars)}
itos[0] = '.'

In [4]:
# Create Dataset
xs, ys = [], []
for name in names:
    chs = ['.'] + list(name) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        xs.append(stoi[ch1])
        ys.append(stoi[ch2])
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()

In [5]:
# initialize netowrk
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27,27), generator=g, requires_grad=True)


In [10]:
# Gradient Descent
for k in range(101):
    # forward pass
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = torch.matmul(xenc, W)
    # Softmax
    #counts = logits.exp()
    #probs = counts / counts.sum(dim=1, keepdim=True)
    #loss = -probs[torch.arange(num), ys].log().mean()
    loss = F.cross_entropy(logits, ys)
    if k % 10 == 0:
        print(f'loss={loss.item()}')

    # backward pass
    W.grad = None
    loss.backward()

    # update
    W.data += -50 * W.grad

loss=2.459230422973633
loss=2.4590423107147217
loss=2.458867073059082
loss=2.458702802658081
loss=2.4585487842559814
loss=2.4584035873413086
loss=2.4582672119140625
loss=2.4581384658813477
loss=2.4580163955688477
loss=2.4579017162323
loss=2.4577927589416504


In [14]:
# Generate some names
for i in range(5):
    out = []
    ix = 0
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        logits = torch.matmul(xenc, W)
        counts = logits.exp()
        p = counts / counts.sum()

        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        if ix == 0:
            break
        out.append(itos[ix])
    print(''.join(out))

s
ki
olyvitay
ren
balyduwpri
